In [2]:
import tensorflow as tf
import numpy as np
import importlib 
import preprocessing
importlib.reload(preprocessing)
from preprocessing import *
import warnings
warnings.filterwarnings('ignore')

In [3]:
X, y = import_data()
X_train,X_test,y_train,y_test = train_test_total(X, y)

In [4]:
def rnn_model(features, labels, mode, params):
  
  
  num_hidden_layers = len(params['hidden_layers'])  
  
  outputs = features["x"]  

  model = params['model']
    
  activation = params.get("activation", tf.nn.tanh)

  print("Using activation - ", activation)

  for i in range(num_hidden_layers):  

    
      cell = model(name = 'cell'+str(i), num_units = params['hidden_layers'][i])
      outputs, state = tf.nn.dynamic_rnn(cell=cell,
                                       inputs=outputs,
                                       dtype=tf.float64)
    
      #outputs = tf.layers.batch_normalization(outputs)
        
      outputs = activation(outputs)
      outputs = tf.nn.dropout(outputs, params['dropout'])

  #FLatten the output of LSTM layers
  outputs = tf.contrib.layers.flatten(outputs) 

  # FC Layer
  logits = tf.layers.dense(inputs=outputs, units=params['num_classes'])

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.AdamOptimizer(learning_rate=params['learning_rate'])
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
    


In [5]:
def calc(eeg_classifier, X_train, y_train, X_test, y_test):

    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_train},
        y=y_train,
        batch_size=50,
        num_epochs=20,
        shuffle=True)

    eeg_classifier.train(
        input_fn=train_input_fn,
        steps=100)

    eval_test_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_test},
        y=y_test,
        num_epochs=1,
        shuffle=False)

    eval_train_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_train},
        y=y_train,
        num_epochs=1,
        shuffle=False)

    train_results = eeg_classifier.evaluate(input_fn=eval_train_fn)
    test_results = eeg_classifier.evaluate(input_fn=eval_test_fn)
    print(train_results)
    print(test_results)

In [ ]:
activations = [tf.nn.relu, tf.nn.crelu, tf.nn.elu, tf.nn.selu, tf.nn.softplus, tf.nn.softsign,
               tf.sigmoid, tf.tanh]

for act in activations:
    
    print("="*75)
    print('Trying out activation - ', str(act))
    
    eeg_classifier = tf.estimator.Estimator(model_fn=rnn_model,
                                            params = {'hidden_layers' : [64, 64], 'num_classes' : 4, 
                                                      'learning_rate' : 0.001, 'model' : tf.nn.rnn_cell.LSTMCell,
                                                     'dropout' : 0.5, 'activation' : act})


    calc(eeg_classifier, X_train, y_train, X_test, y_test)